Ссылка на данный блокнот: https://colab.research.google.com/drive/1Z5fVuajC1rLZQe_w2rJ2LFEzh1SV6KXq?usp=drive_link

Устанавка необходимых модулей, как и в файле "Generation_files.ipynb" для работы Spark.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark

findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import Window
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Объявление схемы данных для дальнейшего считывания JSON файлов и составления Data Frame.

In [ ]:
schema = T.StructType([T.StructField("INN", T.StringType(), True),
                       T.StructField("raw_cookie", T.ArrayType(T.MapType(T.StringType(), T.StringType()))),
                       T.StructField("event_type", T.StringType(), True),
                       T.StructField("event_action", T.StringType(), True),
                       T.StructField("data_value", T.StringType(), True),
                       T.StructField("geocountry", T.StringType(), True),
                       T.StructField("city", T.StringType(), True),
                       T.StructField("user_os", T.StringType(), True),
                       T.StructField("systemlanguage", T.StringType(), True),
                       T.StructField("geoaltitude", T.StringType(), True),
                       T.StructField("meta_platform", T.StringType(), True),
                       T.StructField("screensize", T.StringType(), True),
                       T.StructField("timestampcolumn", T.DateType(), True)
                       ])

Функции "read_file_JSON" считывает JSON файлы (закомментированный код предназначен для удаления дубликатов, при оставлении последнее изменения в данных), на вход данной функции первым и обязательным аргументом передается схема данных, которая содержится в JSON файлах, соответственно данная функция будет являться универсальной для любых JSON файлом, вторым аргументом можно передать целое число, которое обозначает какое количество последних файлов считать, если оставить значение пустым данные считаются за все время.

In [ ]:
path_to_files = "/content/gdrive/MyDrive/data/json/"
filenames = sorted(os.listdir(path_to_files), reverse=True)


def read_file_JSON(schema_json_file: T.StructType, count: int = len(filenames)):
    blank_DF = spark.createDataFrame([], schema=schema_json_file)
    for i in filenames[:count]:
        table = spark.read.format("json") \
            .load(f"{path_to_files}{i}", schema=schema_json_file)
        blank_DF = (blank_DF.union(table)
                    #(узнать как удалить дубликаты, оставив последний)
                    #  .withColumn('rank',
                    #              F.rank().over(Window.partitionBy('INN')
                    #              .orderBy(F.desc('timestampcolumn'))))
                    #  .filter('rank = 1')
                    #  .drop('rank')
                    )
    return blank_DF


def search_values_from_cookie(key):
    return f"filter(raw_cookie, x -> x.key='{key}')"


@F.udf
def match_code(x):
    return [None, "IDFA", "GAID"][1 if x == "IOS" else 2 if x == "Android" else 0]


# Создание витрины "A".
В общем и целом данная витрина - это наша сгенерированная таблица, только без поля "INN".

In [ ]:
data_mart_A = read_file_JSON(schema).drop("INN")
data_mart_A.show(5)

+--------------------+----------+---------------+--------------------+------------------+-------------+------------+--------------+--------------------+-------------+----------+---------------+
|          raw_cookie|event_type|   event_action|          data_value|        geocountry|         city|     user_os|systemlanguage|         geoaltitude|meta_platform|screensize|timestampcolumn|
+--------------------+----------+---------------+--------------------+------------------+-------------+------------+--------------+--------------------+-------------+----------+---------------+
|[{value -> SA1.01...|  REGISTER|login-check-otp|                null|Dominican Republic|Santo Domingo|BlackBerryOS|            RU|     19,-70.66666666|       MOBAIL| 1920x1080|     2024-02-08|
|[{value -> SA1.13...|  REGISTER|       pageview|                null|           Bahrain|       Manama|     Windows|            RU|            26,50.55|          WEB| 1920x1080|     2024-02-08|
|[{value -> SA1.5d...|    SUBM

# Создание витрины "В".
В данной витрине необходимо найти все "INN" пользователей. Для поля "ID" в данной витрине была примена оконная функция, с целью показать свои знания, в последующих витриниках оконных функций использовано не будет из-за ее долгой работы при сортировке данных.

In [ ]:
# Создать цикл счтывания DF по датам. удалять дубликать на объединении дней, удалять cookie по INN

data_mart_B = read_file_JSON(schema).select("INN")
data_mart_B = data_mart_B.select(F.row_number()
                                 .over(Window.orderBy(data_mart_B.INN.desc()))
                                 .alias("ID"), "INN")
data_mart_B.show(5)


+---+------------+
| ID|         INN|
+---+------------+
|  1|999766762140|
|  2|999747435335|
|  3|999079527236|
|  4|998831247489|
|  5|998700646665|
+---+------------+
only showing top 5 rows



# Создание витрины "С"
В данной витрине необходимо собрать куки сайта. Для это использовалась функция
"pyspark.sql.functions.expr", в которую передовалось сгенирированное значение из функции "search_values_from_cookie". Для генерации ID используется метод "monotonically_increasing_id". В последующий витринах используются эти функции.

In [ ]:
data_mart_C = read_file_JSON(schema)
data_mart_C = data_mart_C.select((
    F.monotonically_increasing_id() + 1).alias("ID"),
    F.expr(search_values_from_cookie("_sa_cookie_a"))[0]["value"]
                                  .alias("sa_cookie_a"))
data_mart_C.show(5, truncate=False)

+-----------+---------------------------------------------------+
|ID         |sa_cookie_a                                        |
+-----------+---------------------------------------------------+
|17179869185|SA1.0184e060-8cb1-49e2-838f-01a52e5fc71f.3447625739|
|17179869186|SA1.13832ff4-4a45-46bf-a467-0ad428696a9a.2950859678|
|17179869187|SA1.5d116c66-ee19-401f-a50b-a2ee421c5d6f.6999480288|
|17179869188|SA1.8ad157ab-7751-4b7a-a6c4-cf6ccf00673b.4530866796|
|17179869189|SA1.44abfe17-859c-4883-93f4-8ecf7a59b8d1.7556060237|
+-----------+---------------------------------------------------+
only showing top 5 rows



# Создание витрины "D"
Тоже самое что и в витрине "В". Подразумевается что запущен файл "Generation_files.ipynb", который к этому моменту, должен сгенерировать дополнительные JSON файлы, которые не пересекутся с витриной "В" при объединении.

In [ ]:
data_mart_D = read_file_JSON(schema).select("INN")
data_mart_D = data_mart_D.select((F.monotonically_increasing_id() + 1)
                         .alias("ID"), "INN")
data_mart_D.show(5, truncate=False)

+-----------+------------+
|ID         |INN         |
+-----------+------------+
|17179869185|388787686306|
|17179869186|275323062033|
|17179869187|694374478861|
|17179869188|976816510709|
|17179869189|884113988041|
+-----------+------------+
only showing top 5 rows



# Создание витрины "E"
Необходимо найти телефон польхователя и захешировать его алгоритмом хешировани **md5**. Делается это с помощью встроеной функции "pyspark.sql.functions.md5"

In [ ]:
data_mart_E = read_file_JSON(schema).select(
    (F.monotonically_increasing_id() + 1).alias("ID"),
    F.md5(F.expr(search_values_from_cookie("user_phone"))[0]["value"])
    .alias("hash_phone_md5"))

data_mart_E.show(5, truncate=False)


+-----------+--------------------------------+
|ID         |hash_phone_md5                  |
+-----------+--------------------------------+
|17179869185|87db8810f0fcaa5cc6556bb9d541fb42|
|17179869186|1e33bbb442ee6a3fee2aae2de60d6cf9|
|17179869187|dce6177a2b3a8c984b69e170904b914e|
|17179869188|8f60a25ae722b1f974d8cce2d61d0b0f|
|17179869189|c8b128c408c8eb7ae05feedfcfba95b4|
+-----------+--------------------------------+
only showing top 5 rows



# Создание витрины "F"
Тоже самое что и в витрине "E", только с полем "user_mail".

In [ ]:
data_mart_F = read_file_JSON(schema).select(
    (F.monotonically_increasing_id() + 1).alias("ID"),
    F.md5(F.expr(search_values_from_cookie("user_mail"))[0]["value"])
    .alias("hash_email_md5"))

data_mart_F.show(5, truncate=False)

+-----------+--------------------------------+
|ID         |hash_email_md5                  |
+-----------+--------------------------------+
|17179869185|4b1acf39868bedb5e81a592404ea4acf|
|17179869186|2620fe26a57e2b76d49bb4205a3e4c71|
|17179869187|4250d7ae810ede6339746fc0411955d2|
|17179869188|36e929c898713524a761cd9fc6054bb8|
|17179869189|c4a13118253dd80f0d150e3a03442542|
+-----------+--------------------------------+
only showing top 5 rows



# Создание витрины "G"

Замена значения через функцию "match_code".

In [ ]:
data_mart_G = read_file_JSON(schema)

data_mart_G = data_mart_G.select(
    (F.monotonically_increasing_id() + 1).alias("ID"),
    F.expr(search_values_from_cookie("user_uid"))[0]["value"].alias("user_uid"),
    match_code("user_os").alias("match_code"))

data_mart_G.show(5, truncate=False)

+-----------+--------+----------+
|ID         |user_uid|match_code|
+-----------+--------+----------+
|17179869185|1324087 |null      |
|17179869186|4987999 |null      |
|17179869187|3249335 |GAID      |
|17179869188|9593724 |null      |
|17179869189|0742701 |null      |
+-----------+--------+----------+
only showing top 5 rows



# Создание обобщенной витрины объединяющая все предыдущие витрины



In [ ]:
df_A = data_mart_A.alias("A")
df_B = data_mart_B.alias("B")
df_C = data_mart_C.alias("C")
df_D = data_mart_D.alias("D")
df_E = data_mart_E.alias("E")
df_F = data_mart_F.alias("F")
df_G = data_mart_G.alias("G")

In [ ]:
main_marts = df_A.join(df_D, on=(
        (F.md5(df_D.INN) == df_A.data_value) |
        (F.sha2(df_D.INN, 256) == df_A.data_value))) \
    .select(df_A.raw_cookie,
            df_D.ID.alias("ID_D"),
            df_D.INN.alias("INN_D"),
            df_A.data_value) \
    .join(df_B, on=(
        (F.md5(df_B.INN) == df_A.data_value) |
        (F.sha2(df_B.INN, 256) == df_A.data_value)), how="left") \
    .select("raw_cookie",
            "ID_D",
            "INN_D",
            df_B.ID.alias("ID_B"),
            "data_value")


In [ ]:
main_marts.join(df_E,
                F.md5(F.expr(search_values_from_cookie("user_phone"))[0]["value"]) ==
                                                          df_E.hash_phone_md5, "left") \
    .join(df_F,
          F.md5(F.expr(search_values_from_cookie("user_mail"))[0]["value"]) ==
                                                    df_F.hash_email_md5, "left") \
    .join(df_G,
          F.expr(search_values_from_cookie("user_uid"))[0]["value"] ==
                                                 df_G.user_uid, "left") \
    .join(df_C,
          F.expr(search_values_from_cookie("_sa_cookie_a"))[0]["value"] ==
                                          data_mart_C.sa_cookie_a, "left") \
    .select(data_mart_G.user_uid,
            main_marts.INN_D.alias("INN"),
            F.expr(search_values_from_cookie("user_phone"))[0]["value"].alias("user_phone"),
            F.expr(search_values_from_cookie("user_mail"))[0]["value"].alias("user_email"),
            main_marts.data_value.alias("inn_hash"),
            df_E.hash_phone_md5,
            df_F.hash_email_md5,
            F.expr(search_values_from_cookie("org_uid"))[0]["value"].alias("org_uid"),
            main_marts.ID_B,
            df_C.ID.alias("ID_C"),
            main_marts.ID_D,
            df_E.ID.alias("ID_E"),
            df_F.ID.alias("ID_F"),
            df_G.ID.alias("ID_G"),
            F.array(F.expr(search_values_from_cookie("_sa_cookie_a"))[0]["value"],
                    F.expr(search_values_from_cookie("_fa_cookie_a"))[0]["value"],
                    F.expr(search_values_from_cookie("_ym_cookie_c"))[0]["value"],
                    F.expr(search_values_from_cookie("_fbp"))[0]["value"]).alias("array_coockie")) \
    .show(5, truncate=False)


+--------+------------+---------------+------------------+----------------------------------------------------------------+--------------------------------+--------------------------------+-------+----+------------+------------+------------+------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+
|user_uid|INN         |user_phone     |user_email        |inn_hash                                                        |hash_phone_md5                  |hash_email_md5                  |org_uid|ID_B|ID_C        |ID_D        |ID_E        |ID_F        |ID_G        |array_coockie                                                                                                                          |
+--------+------------+---------------+------------------+----------------------------------------------------------------+--------------------------------+--------------------------------+---